In [1]:
#pip install cmake 'gym[atari]' scipy

In [2]:
import gym 

In [3]:
env = gym.make("Taxi-v3").env
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [4]:
# An action number from 0-5 where:
# 0 = south
# 1 = north
# 2 = east
# 3 = west
# 4 = pickup
# 5 = dropoff

In [5]:
env.reset()  
env.render()

#print("Number of action space {}".format(env.action_space))
#print("Number of state sSpace {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [6]:
state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)
 
env.s = state
env.render()

State: 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [7]:
env.P[100]
# action -> probability, nextstate, reward, done

{0: [(1.0, 200, -1, False)],
 1: [(1.0, 0, -1, False)],
 2: [(1.0, 120, -1, False)],
 3: [(1.0, 100, -1, False)],
 4: [(1.0, 100, -10, False)],
 5: [(1.0, 100, -10, False)]}

Solving without RL

In [8]:
env.s = 328 

epochs = 0
penalties, reward = 0, 0
frames = [] 
done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)
    if reward == -10:
        penalties += 1
    frames.append({'frame': env.render(mode='ansi'), 'state': state, 'action': action, 'reward': reward })
    epochs += 1
    
    
print("Step taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Step taken: 86
Penalties incurred: 19


In [12]:
from IPython.display import clear_output 
from time import sleep 

def print_frames(frames): 
    for i, frame in enumerate(frames): 
        clear_output(wait = True) 
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(0.1) 

#print_frames(frames)

In our Taxi environment, we have the reward table, ```P```, that the agent will learn from. It does thing by looking receiving a reward for taking an action in the current state, then updating a Q-value to remember if that action was beneficial.

Q-values are initialized to an arbitrary value, and as the agent exposes itself to the environment and receives different rewards by executing different actions, the Q-value are updated using the equation: 

$Q(state, action) \leftarrow (1 - \alpha)Q(state, action) + \alpha(reward + \gamma*max_{a}Q(\text{next state}, \text{all actions}))$


In [13]:
import numpy as np 

q_table = np.zeros([env.observation_space.n, env.action_space.n])

print(q_table)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


In [14]:
import random 

alpha = 0.1 
gamma = 0.6 

epsilon = 0.1

all_epochs = [] 
all_penalties = [] 

for i in range(1, 100000):
    state = env.reset() 

    epochs, penalties, reward = 0, 0, 0 
    done = False 

    while not done:
        if random.uniform(0, 1) < epsilon: 
            action = env.action_space.sample() 
        else: 
            action = np.argmax(q_table[state])

        next_state, reward, done, info = env.step(action) 

        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])

        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max) 
        q_table[state, action] = new_value 

        if reward == -10: 
            penalties += 1

        state = next_state 
        epochs += 1

    if i % 100 == 0: 
        clear_output(wait = True) 
        print(f"Episode: {i}")

print("OK")

Episode: 99900
OK


In [15]:
q_table[328]

array([ -2.40604837,  -2.27325184,  -2.40099822,  -2.35938433,
       -10.65925005, -10.96163233])

In [16]:
total_epochs, total_penalties = 0, 0
episodes = 10

frame = [] 
min_pen = 9999999999999

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    ff = []
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)
           
        if reward == -10:
            penalties += 1

        ff.append({'frame': env.render(mode='ansi'), 'state': state, 'action': action, 'reward': reward })

        epochs += 1
    
    if min_pen > penalties: 
        min_pen = penalties 
        frame = ff 

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 10 episodes:
Average timesteps per episode: 11.5
Average penalties per episode: 0.0


In [17]:
#print_frames(frame)